Generate an OSPC tree from the Slovenian and US BBss isolates from Klemen and Ira
April 25 2021
lemieux@broadinstitute.org

In [1]:
import pandas as pd
from Bio import SeqIO
import os

In [2]:
#pres_abs = pd.read_csv("../roary/Bbss_combined/spades_assem_4_9_2021/gene_presence_absence.csv")

In [3]:
pres_abs = pd.read_csv("../results/roary-results/gene_presence_absence.csv")

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (17,18,22,24,25,26,27,50,53,57,76,79,83,89,91,94,99,109,110,111,123,125,127,129,132,139,141,147,151,154,162,169,185,186,191,199,201,202,203,204,206,211,213,214,217,220,222,228,230,242,255,257,259,268,270,274,277,278,279,281,284,293,294,309,312) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
pres_abs

,Gene,Non-unique Gene name,Annotation,No. isolates,No. sequences,Avg sequences per isolate,Genome Fragment,Order within Fragment,Accessory Fragment,Accessory Order with Fragment,...,UWI274,UWI275,UWI276,UWI277,UWI278,UWI279,UWI280,UWI281,UWI282,UWI283
0,mvaA,NaN,3-hydroxy-3-methylglutaryl-coenzyme A reductase,299,300,1.01,1,1995,NaN,NaN,...,EIKACIID_00457,BDNEAJCL_00441,BMBJHJKP_01099,CGOHJBJA_00691,OMNPMPEN_00203,PLAGJJIO_00453,NFDFKCDH_00438,NABAPFHD_00439,AJDJAMLE_00120,JOFNOGJM_00751
1,tufA,NaN,Elongation factor Tu,299,299,1.00,1,1227,NaN,NaN,...,EIKACIID_00243,BDNEAJCL_00388,BMBJHJKP_01030,CGOHJBJA_00477,OMNPMPEN_00417,PLAGJJIO_00239,NFDFKCDH_00490,NABAPFHD_00608,AJDJAMLE_00334,JOFNOGJM_00177
2,apt,NaN,Adenine phosphoribosyltransferase,299,300,1.01,1,2087,NaN,NaN,...,EIKACIID_00551,BDNEAJCL_00279,BMBJHJKP_00492,CGOHJBJA_00785,OMNPMPEN_00776,PLAGJJIO_00547,NFDFKCDH_00626,NABAPFHD_00744,AJDJAMLE_00025,JOFNOGJM_00605
3,rnz,NaN,Ribonuclease Z,299,300,1.01,1,2064,NaN,NaN,...,EIKACIID_00528,BDNEAJCL_00624,BMBJHJKP_00524,CGOHJBJA_00762,OMNPMPEN_00118,PLAGJJIO_00524,NFDFKCDH_00154,NABAPFHD_00511,AJDJAMLE_00049,JOFNOGJM_00582
4,group_1106,NaN,hypothetical protein,299,300,1.01,1,1985,NaN,NaN,...,EIKACIID_00447,BDNEAJCL_00431,BMBJHJKP_00723,CGOHJBJA_00681,OMNPMPEN_01054,PLAGJJIO_00443,NFDFKCDH_00428,NABAPFHD_00449,AJDJAMLE_00130,JOFNOGJM_00741
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3189,group_758,NaN,hypothetical protein,1,4,4.00,1,86,352.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3190,group_759,NaN,hypothetical protein,1,4,4.00,1,241,353.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3191,group_996,NaN,hypothetical protein,1,3,3.00,134,1,354.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3192,group_997,NaN,hypothetical protein,1,3,3.00,1,361,525.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
to_match = "Outer surface protein C"

In [6]:
ospC = pres_abs.loc[pres_abs["Annotation"] == to_match, :]

In [7]:
ospC["No. isolates"].sum()

313

There are two ways to get a list of ospC proteins: 
    1. open the annotated prokka files and extract ones with descriptions that are annotated as OspC
    2. open the annotated prokka files and extract ones with names corresponding to roary clusters
will implement the first one for now.

In [8]:
colnames = list(ospC)

In [9]:
sample_names = colnames[15:]

In [10]:
sample_names = [i.split("_")[0] for i in sample_names]

In [12]:
#sample_names.remove('UNY252') # note sure why this is missing from our google sheets DB

In [18]:
sample_names[0:10]

['ESI28',
 'ESI284',
 'ESI285',
 'ESI286',
 'ESI287',
 'ESI288a',
 'ESI289',
 'ESI290',
 'ESI291',
 'ESI292']

In [41]:
records = []
for sample_name in sample_names:
    counter = 0
    for record in SeqIO.parse("../results/prokka/" + sample_name + "/prokka_annotated.faa", "fasta"):
        if "Outer surface protein C" in record.description:
            counter += 1
            record.id = sample_name + "_" + str(counter) + "_" + record.id + "_length_" + str(len(record.seq))
            record.description = ""
            if len(record.seq) > 190:
                records.append(record)

In [42]:
#SeqIO.write(records, "/Users/lemieux/Library/CloudStorage/Box-Box/Borrellia_collaboration/alignments/OspC_alignment_fasta.csv", "fasta")
SeqIO.write(records, "../results/alignments/OspC_alignment_fasta.csv", "fasta")

290

In [43]:
len(sample_names)

298

In [46]:
# align with mafft
os.system("mafft ../results/alignments/OspC_alignment_fasta.csv > ../results/alignments/OspC_alignment_alignment_5_3_2023.fasta")

0

In [49]:
# construct tree with fasttree
os.system("fasttree -gtr ../results/alignments/OspC_alignment_alignment_5_3_2023.fasta > ../results/trees/OspC_full_length_5_3_2023.nwk")

0

To do (April 24 2021):
- add in osp C type information
- obtain osp C type information from Ira